In [26]:
import pandas as pd
import numpy as np
import polars as pl

# Imports

In [18]:
file_path_forecasts = r"C:\Users\mariu\Downloads\dataset2014-2022-v3.csv"
file_path_company = r"C:\Users\mariu\Downloads\dataset-company-industry-data-v1.csv"

df = pd.read_csv(file_path_forecasts)
df2 = pd.read_csv(file_path_company)

# Merge

In [41]:
def join_fundamentals(df,df2):
    df = df.rename(columns={col: col.lower() for col in df.columns})
    
    result = df.merge(df2[["tic", "sic"]], how="left", left_on="oftic", right_on="tic")
    
    num_join_fails = result["sic"].isna().sum()
    
    print(f"The number of failed merges is: {num_join_fails}")
    return result

In [42]:
merged_df = join_fundamentals(df,df2)

The number of failed merges is: 255745


In [43]:
merged_df.head()

,ticker,cusip,oftic,cname,estimator,analys,fpi,measure,value,fpedats,revdats,anndats,actual,anndats_act,tic,sic
0,0000,87482X10,TLMR,TALMER BANCORP,2500,72481,1,EPS,1.20,2014-12-31,2014-12-03,2014-12-03,1.21,2015-01-30,TLMR,6020.0
1,0000,87482X10,TLMR,TALMER BANCORP,2500,72481,1,EPS,1.20,2014-12-31,2014-12-03,2014-12-03,1.21,2015-01-30,TLMR,6020.0
2,0000,87482X10,TLMR,TALMER BANCORP,2500,72481,1,EPS,1.20,2014-12-31,2014-12-03,2014-12-03,1.21,2015-01-30,TLMR,6020.0
3,0000,87482X10,TLMR,TALMER BANCORP,2500,72481,1,EPS,1.20,2014-12-31,2014-12-03,2014-12-03,1.21,2015-01-30,TLMR,6020.0
4,0000,87482X10,TLMR,TALMER BANCORP,2500,72481,1,EPS,0.85,2015-12-31,2015-02-18,2015-02-18,1.02,2016-01-26,TLMR,6020.0


# Data set restrictions

Restrictions should be implemented with respective functions

- Sample horizon 10 - 20 years
- Frequency: annual
- Include only forecasts issued no earlier than 1 year ahead and no later than 30 days before fiscal year end
- Exclude forecasts for companies which only one analyst provides a forecast
- Exclude all analysts appearing in the initial 2-3 years of the dataset (migitating problem of experience)
- (Retain the last forecast an analyst issues in a particular year)
- Drop all unneccessary columns from df
- ...

# DV and Feature Generation

Feature list (Proxies for analyst characteristics):

- Forecasting ability/experience: number of years for which analyst i supplied at least one forecast during the first 11 months of the year through year t
- Broker size: a dummy variable set to 1 if analyst i is employed by a firm in the top size decile during year t, and set to 0 otherwise. Size deciles are calculated based on the number of analysts employed in year t.
- Complexity: number of firms for which analyst i supplied at least one forecast during the first 11 months of year t OR number of two-digit SICs for which analyst i supplied at least one forecast during the first 11 months of year t minus the average number of two-digit SICs followed by an analyst following firm j at time t.
- Forecast age: age (in days) of analyst i's forecast for firm j's earnings at time t
- 


In [4]:
df["VALUE"].max()

4821600.0

In [15]:
df["ANALYS"].isna().sum()


0